##### Canned question: Endpoints from clinical trials

This is a simple demo showing how an LLM can summarize clinical trial features (such as endpoints) from the supporting trials listed in Translator results, enabling users to compare trial features side by side.

In [3]:
from clinical_trials_isb_client import get_clinical_trials_isb_info
from ollama import Client
from driver import load_trapi_response
import json
import summarizers.trapi_tools as tools

In [5]:
trapi_path = 'data/101635ed-6bb7-46fc-a2f1-96d814c34040.json'
orig = load_trapi_response(trapi_path)
kg_edges = orig['fields']['data']['message']['knowledge_graph']['edges']
uniq = tools.extract_unique_edges(kg_edges.values())
print(f"There are {len(uniq)} unique edges in this response.")
ct_ids= []
for edge in uniq:
    ids = tools.extract_clinical_trials(edge)
    ct_ids.extend(ids)
ct_ids = list(set(ct_ids))
print(f"Return {len(ct_ids)} clincial trial IDs.")

There are 1344 unique edges in this response.
Return 3 clincial trial IDs.


In [7]:
# fetch the clinical trial objects from ISB API
# TODO: discuss with the auther - results data like stats data was not included

ct_info = await get_clinical_trials_isb_info(ct_ids)

In [2]:
# ct_info

In [91]:
# TODO: What are the interesting, useful, or frequently asked follow-up questions once endpoints are identified?  

def generate_prompt(cts):
    if len(cts) > 0: 
#         prompt = f"""Please summarize the primary and key secondary endpoints in the following clinical trials using concise bullet points. For each endpoint, note whether the trial achieved it based on reported results (include effect sizes if available). If an endpoint result is not reported, indicate ‘not reported.’ Present each trial in a standardized format with two sections: (1) Endpoints and (2) Results.
# {cts}
#         """
        prompt = f"""Please summarize the following clinical trials in a concise and standardized format. For each trial, present the information in three sections using concise bullet points:
1. Trial Summary
- Trial type
- Number of participants
- Summarize the trial in a concise way

2. Endpoints
- Primary endpoint(s)
- Key secondary endpoint(s)

3. Results
- State whether each endpoint was achieved based on reported results
- Include effect sizes if available
- If results are not reported for an endpoint, state “not reported”

Clinical Trails:
{cts}
"""        
    else:
        prompt = None

    return prompt

In [93]:
print(generate_prompt(ct_info.get('studies', None)))

Please summarize the following clinical trials in a concise and standardized format. For each trial, present the information in three sections using concise bullet points:
1. Trial Summary
- Trial type
- Number of participants
- Summarize the trial in a concise way

2. Endpoints
- Primary endpoint(s)
- Key secondary endpoint(s)

3. Results
- State whether each endpoint was achieved based on reported results
- Include effect sizes if available
- If results are not reported for an endpoint, state “not reported”

Clinical Trails:
[{'protocolSection': {'outcomesModule': {'secondaryOutcomes': [{'measure': 'Improvement in Predictive Accuracy of Lung Nodule Diagnosis Using the C Statistic (Area Under the Curve, AUC)', 'time_frame': 'Through study completion, an average of 2 years and 6 months', 'description': 'The predictive accuracy of the lung nodule diagnosis model was assessed using the C statistic, also referred to as the area under the receiver operating characteristic (ROC) curve (AUC)

In [95]:
client = Client()
prompt = generate_prompt(ct_info.get('studies', None))
# print(prompt)
if prompt:
    messages = [
      {
        'role': 'user',
        'content': prompt,
      },
    ]
    
    response = client.chat(
        model='gpt-oss:20b',
        messages=messages,
        options={'num_ctx': 8192},  # 8192 is the recommended lower limit for the context window
      )

    response_content = response['message']['content']
else:
    response_content = "There isn't clinical trial information in the result."

print(response_content)

**Trial 1 – NCT03824535**  
*18F‑FSPG PET/CT in Diagnosing Early Lung Cancer in Patients With Lung Nodules*  

**Trial Summary**  
- **Trial type:** Diagnostic, Phase II, Interventional, single‑group.  
- **Participants:** Number not reported.  
- **Brief summary:** Evaluates the ability of 18F‑FSPG PET/CT to discriminate benign from malignant pulmonary nodules, comparing it to standard 18F‑FDG PET/CT.  

**Endpoints**  
- **Primary:** Sensitivity, specificity, PPV, NPV, and overall accuracy of ultrasonography and capnography for malignant nodule detection (note: outcome metrics appear mismatched to the imaging modality described).  
- **Key secondary:** Improvement in predictive accuracy measured by the C‑statistic (AUC).  

**Results**  
- Primary outcome: **not reported**.  
- Secondary outcome: **not reported**.  


---  

**Trial 2 – NCT05848635**  
*Hydromorphone for Perioperative Analgesia in Lung Tumor Ablation Parallel Controlled*  

**Trial Summary**  
- **Trial type:** Treat